# Dissertation Analysis

## 1、Hidden Voice Commands

### (1) Background Introduction
This section mainly introduces normal ASR process. (Automatic Speech Recognition)
<img src="Pic1.png"/>
**Pre-processing:**  
A rough processing procedure,  
only filters out frequencies that are beyond the range of a human voice  
and eliminates time periods where the signal energy falls below a particular threshold.   
  
**Feature Extraction:**  
splits the filtered audio signal into short (usually around 20 ms) frames  
and extracts **MFCC** features from each frame.  
  
**Model-Based Prediction:**  
takes as input the extracted features,  
and matches them against an existing model built offline to generate text predictions.  
  
**Post-processing:**  
ranks the text predictions by employing additional sources of information,  
such as grammar rules or locality of words

### (2) Black-Box Attacks
This section describes the general approach of generating a black-box attack. 
<img src="Pic2.png"/>
**Step One:**  
The attacker first produces a normal command that he wants to executed on the targeted device.  
He should ensure this command can be transcribed correctly, and then he can take the command as an input.  
  
**Step Two:**  
The **audio mangler**, shown as the greybox in the figure,  
firstly extract MFCC features according to your starting set of parameters,  
then excute a inverse-MFCC to transform the obtained acoustic features back to audio file (we call this obfuscated audio).  
beacuse of the extraction process, the origin audio lose lots of infomation and introduce some noise,  
resulting in difficulties for human understanding.  
  
**Step Three:**  
Since the attacker does not know the MFCC features used by the speech recognition system,  
he need to put the obfucated audio into ASR to verify whether it can be successfully transcribed.
if the ASR correctly understand our origin input, it prove that our MFCC parameters setting is similar to ASR's.  
  
**Step four:**  
Final step, we need to ensure the crime can't understand the attack command.  
we take the obfuscated audio from step3, if we can easily understand it, we discard it and adjust our parameters setting.  
  
**Till here**, we produce the candidate obfuscated command, which is recognized by machine but not by human,  
can be used to conduct a actual voice attack.

### (3) Denfense Strategy
In this section, we will discuss some strategy to avoid voice attack.  
  
**Defenses that notify**  
The “Beep”, the “Buzz” and the “Lightshow”.  
this part mainly analyse the effcacy and practicability of voice warning, vibration warning and light warning.  
  
**Defenses that challenge**  
Using a voice authentication system to confirm the command.  
  
**Defenses that detect and prevent**  
Speaker recognition, the filter, the classifier.  

## 2、Exploiting the Gap between Human and Machine Speech Recognition
This dissertation futher clarifies all experimental steps in detail.

**MFCC Introduction:**  
Mel-frequency cepstral coefficients (MFCCs) are used to represent the short-term power spectrum of audio on a nonlinear mel frequency scale.  
MFCC is based on human hearing perceptions with frequency bands, equally imitate the process of human understanding of sound.  
  
  
**MFCC Parameters:**  
MFCC computation requires various parameters to be specified.    
We focus on four independent parameters: wintime, hoptime, numcep and nbands.   
**wintime** determines the length of the timeframe over which the signal is considered as being statistically constant;  
**hoptime** determines the size of the time step between successive windows;  
**numcep** is the number of cepstra, i.e., the number of coefficients to output;  
**nbands** is the number of warped spectral bands to use while aggregating energy levels for closely spaced frequencies.
  
In this part, We will experimentally observe the effect of changing each of these parameters independently on the perceived
quality of mangled audio output.  
  
  
**Feature extraction with tuned MFCC parameters**  
After experimentally determining the range of MFCC parameter values, acoustic features are extracted by computing MFCCs of the input signal using the chosen MFCC parameters. Computing MFCCs is lossy: the process considers the signal to be statistically constant over a small time window and also aggregates the energy levels of closely spaced frequencies to represent the total energy in various frequency regions on the mel frequency scale.  
  
We use the tuned MFCC parameters to compute MFCCs of an unmodified audio command. The resulting MFCCs contain just enough acoustic information, ensured by careful selection of MFCC parameters, such that a mangled audio signal reconstructed
from them will be correctly recognized by the targeted speech recognition system.  
  
**Inverse MFCC**  
The extracted audio features represented as MFCCs are converted back to an audio signal by reversing the steps of the MFCC computation. The inversion of MFCCs back to a waveform involves the addition of noise, since MFCC computation is lossy and aggregates energy levels of closely spaced frequencies into frequency regions. Inversion from MFCCs to audio signals mangles
the original audio signals, making them difficult for human listeners to understand.

## 3、MFCC Computation
This dissertation help me understand the calculation process of MFCC in depth.

### (1) Steps at a Glance
In this section, we will give an overview of the whole computational process.  
  
**Step1:** Frame the signal into short frames.  
**Step2:** For each frame calculate the periodogram estimate of the power spectrum.  
**Step3:** Apply the mel filterbank to the power spectra, sum the energy in each filter.  
**Step4:** Take the logarithm of all filterbank energies.  
**Step5:** Take the DCT of the log filterbank energies.  
**Step6:** Keep DCT coefficients 2-13, discard the rest.  

### (2) Explanation the Meanings of each Step 
**For Step1:**  
An audio signal is constantly changing, so to simplify things we assume that on short time scales the audio signal doesn't change much (when we say it doesn't change, we mean statistically i.e. statistically stationary, obviously the samples are constantly changing on even short time scales). This is why we frame the signal into 20-40ms frames. If the frame is much shorter we don't have enough samples to get a reliable spectral estimate, if it is longer the signal changes too much throughout the frame.  
  
**For Step2:**  
The next step is to calculate the power spectrum of each frame. This is motivated by the human cochlea (an organ in the ear) which vibrates at different spots depending on the frequency of the incoming sounds. Depending on the location in the cochlea that vibrates (which wobbles small hairs), different nerves fire informing the brain that certain frequencies are present. Our periodogram estimate performs a similar job for us, identifying which frequencies are present in the frame.  
  
**For Step3:**  
The periodogram spectral estimate still contains a lot of information not required for Automatic Speech Recognition (ASR). In particular the cochlea can not discern the difference between two closely spaced frequencies. This effect becomes more pronounced as the frequencies increase. For this reason we take clumps of periodogram bins and sum them up to get an idea of how much energy exists in various frequency regions. This is performed by our Mel filterbank: the first filter is very narrow and gives an indication of how much energy exists near 0 Hertz. As the frequencies get higher our filters get wider as we become less concerned about variations. We are only interested in roughly how much energy occurs at each spot. The Mel scale tells us exactly how to space our filterbanks and how wide to make them. See below for how to calculate the spacing.  
  
**For Step4:**   
Once we have the filterbank energies, we take the logarithm of them. This is also motivated by human hearing: we don't hear loudness on a linear scale. Generally to double the percieved volume of a sound we need to put 8 times as much energy into it. This means that large variations in energy may not sound all that different if the sound is loud to begin with. This compression operation makes our features match more closely what humans actually hear. Why the logarithm and not a cube root? The logarithm allows us to use cepstral mean subtraction, which is a channel normalisation technique.  
  
**For Step5:**  
The final step is to compute the DCT of the log filterbank energies. There are 2 main reasons this is performed. Because our filterbanks are all overlapping, the filterbank energies are quite correlated with each other. The DCT decorrelates the energies which means diagonal covariance matrices can be used to model the features in e.g. a HMM classifier. But notice that only 12 of the 26 DCT coefficients are kept. This is because the higher DCT coefficients represent fast changes in the filterbank energies and it turns out that these fast changes actually degrade ASR performance, so we get a small improvement by dropping them.  

### (3) Implementation Steps 
**Mel Scale:**  
<img src="Pic4.png"/>
<img src="Pic3.png"/>
  
**For Step1:**  
Frame the signal into 20-40 ms frames. 25ms is standard. This means the frame length for a 16kHz signal is 0.025*16000 = 400 samples. Frame step is usually something like 10ms (160 samples), which allows some overlap to the frames. The first 400 sample frame starts at sample 0, the next 400 sample frame starts at sample 160 etc. until the end of the speech file is reached. If the speech file does not divide into an even number of frames, pad it with zeros so that it does.  
  
**For Step2:**  
512 points DFT  
hamming window  